<a href="https://colab.research.google.com/github/Andicleomj/Machine-Learning/blob/main/Week11/dummy%20data/dummy%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Andi Cleopatra Maryam Jamila
#### 1103213071
#### Classification dummy data dan buat model classification dengan deep learning

In [2]:
# Import library yang diperlukan
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


* Mengimpor pustaka yang dibutuhkan untuk model deep learning (PyTorch), manipulasi data (NumPy, Pandas), dan pembagian data (train-test split, standardisasi).

In [3]:
# Membuat dummy data untuk klasifikasi
# Fitur X berjumlah 1000 data, 20 fitur
X = np.random.rand(1000, 20)

# Membuat target y yang terdiri dari dua kelas (0 atau 1)
y = np.random.randint(0, 2, size=(1000,))

# Membagi data menjadi train dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalisasi fitur menggunakan StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Konversi data ke format tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


* Membuat data dummy untuk 1000 sampel dan 20 fitur. Target y adalah biner (0 atau 1).
* Data dibagi menjadi set pelatihan dan pengujian (80% untuk pelatihan dan 20% untuk pengujian).
* Data fitur dinormalisasi menggunakan StandardScaler untuk mempercepat konvergensi model.
* Mengonversi data ke format tensor PyTorch agar dapat diproses oleh model.

In [4]:
# Definisi model MLP
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_layers, hidden_units, activation_function):
        super(MLPModel, self).__init__()

        # Menyusun layers berdasarkan parameter eksperimen
        layers = []
        prev_units = input_dim
        for _ in range(hidden_layers):
            layers.append(nn.Linear(prev_units, hidden_units))  # Menambahkan layer Linear
            # Menambahkan fungsi aktivasi sesuai parameter
            if activation_function == 'ReLU':
                layers.append(nn.ReLU())
            elif activation_function == 'Sigmoid':
                layers.append(nn.Sigmoid())
            elif activation_function == 'Tanh':
                layers.append(nn.Tanh())
            elif activation_function == 'Softmax':
                layers.append(nn.Softmax(dim=1))
            prev_units = hidden_units

        layers.append(nn.Linear(prev_units, 2))  # Output layer untuk klasifikasi 2 kelas
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


* Kelas MLPModel adalah model neural network yang dibangun menggunakan nn.Linear untuk setiap layer tersembunyi dan layer output.
* Fungsi aktivasi (ReLU, Sigmoid, Tanh, Softmax) dipilih berdasarkan parameter eksperimen yang diteruskan.
* Model menggunakan Sequential untuk menghubungkan layer-layer tersebut.

In [5]:
# Fungsi untuk melatih model
def train_model(X_train_tensor, y_train_tensor, epochs, learning_rate, batch_size, hidden_layers, hidden_units, activation_function):
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    input_dim = X_train_tensor.shape[1]
    model = MLPModel(input_dim, hidden_layers, hidden_units, activation_function)

    criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss untuk klasifikasi
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        model.train()
        for data, target in train_loader:
            optimizer.zero_grad()  # Reset gradien
            output = model(data)  # Forward pass
            loss = criterion(output, target)  # Hitung loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update parameter model

    return model


* Fungsi train_model melatih model dengan berbagai parameter eksperimen.
* Data pelatihan dimasukkan dalam batch menggunakan DataLoader.
* Fungsi loss yang digunakan adalah CrossEntropyLoss untuk klasifikasi biner.
* Optimizer yang digunakan adalah Adam dengan learning rate yang diberikan.

In [ ]:
# Parameter eksperimen yang akan diuji
epochs = [1, 10, 25, 50, 100, 250]
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]
hidden_layers = [1, 2, 3]
hidden_units = [4, 8, 16, 32, 64]
activation_functions = ['ReLU', 'Sigmoid', 'Tanh', 'Softmax']

best_model = None
best_accuracy = 0

# Melatih dan menguji model dengan berbagai kombinasi parameter
for epoch in epochs:
    for lr in learning_rates:
        for batch_size in batch_sizes:
            for hl in hidden_layers:
                for hu in hidden_units:
                    for af in activation_functions:
                        model = train_model(X_train_tensor, y_train_tensor, epoch, lr, batch_size, hl, hu, af)

                        # Evaluasi akurasi model
                        model.eval()
                        with torch.no_grad():
                            outputs = model(X_test_tensor)
                            _, predicted = torch.max(outputs, 1)
                            accuracy = (predicted == y_test_tensor).sum().item() / y_test_tensor.size(0)
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                best_model = model
                        print(f"Epoch: {epoch}, LR: {lr}, Batch Size: {batch_size}, Hidden Layers: {hl}, Units: {hu}, Activation: {af}, Accuracy: {accuracy}")


Streaming output truncated to the last 5000 lines.
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 8, Activation: Tanh, Accuracy: 0.55
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 8, Activation: Softmax, Accuracy: 0.49
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 16, Activation: ReLU, Accuracy: 0.525
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 16, Activation: Sigmoid, Accuracy: 0.52
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 16, Activation: Tanh, Accuracy: 0.48
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 16, Activation: Softmax, Accuracy: 0.505
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 32, Activation: ReLU, Accuracy: 0.535
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 32, Activation: Sigmoid, Accuracy: 0.515
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 32, Activation: Tanh, Accuracy: 0.5
Epoch: 50, LR: 1, Batch Size: 64, Hidden Layers: 1, Units: 32, Activatio

* Model diuji dengan berbagai kombinasi parameter yang telah didefinisikan.
* Untuk setiap kombinasi, model dilatih dan dievaluasi pada data pengujian.
* Akurasi terbaik dicatat dan model yang memberikan akurasi tertinggi disimpan.

In [ ]:
# Menyimpan model terbaik
torch.save(best_model.state_dict(), 'best_classification_model.pth')


* Setelah menemukan model dengan akurasi terbaik, model tersebut disimpan ke dalam file 'best_classification_model.pth' menggunakan torch.save.